In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para montar Google Drive y acceder a los archivos
from google.colab import drive

# Montar Google Drive para acceder a tus archivos .dta
drive.mount('/content/drive')

# Reemplaza 'ruta/a/tu/carpeta/' con la ruta real en tu Google Drive
# donde están los archivos hh_91.dta y hh_98.dta
ruta_base = '/content/drive/MyDrive/eval/'

# Cargar los datos usando la librería pandas
# Stata: use hh_98.dta, clear
try:
    df_hh98 = pd.read_stata(ruta_base + 'hh_98.dta')
    print("Base de datos 'hh_98.dta' cargada exitosamente.")
except FileNotFoundError:
    print("Error: El archivo 'hh_98.dta' no se encontró. Por favor, verifica la ruta.")
    # Si no tienes el archivo, puedes usar un archivo de ejemplo para que el notebook funcione
    # Por ejemplo, un DataFrame de muestra
    data = {'famsize': [4, 5, 3, 2, 6],
            'educhead': [10, 12, 8, 16, 9],
            'agehead': [45, 55, 30, 60, 42],
            'sexhead': [1, 0, 1, 1, 0],
            'dfmfd': [1, 2, 1, 2, 1],
            'weight': [100, 150, 120, 110, 130]}
    df_hh98 = pd.DataFrame(data)
    print("Se ha creado un DataFrame de ejemplo para continuar.")



In [ ]:
# --- Replicando Comandos Básicos y Descripción de Datos ---

# Stata: describe
print("--- describe ---")
print(df_hh98.info()) # Muestra el tipo de datos y la cantidad de valores no nulos
print(df_hh98.head())  # Muestra las primeras 5 filas (similar a list in 1/5)

# Stata: describe nh-famsize (no es un comando directo en pandas)
# Se hace con df_hh98[['nh', 'famsize']].describe() si estas columnas existen

# Stata: list in 1/3
print("\n--- list in 1/3 ---")
print(df_hh98.head(3)) # Muestra las 3 primeras filas

# Stata: list famsize educhead if (sexhead == 0 & agehead<45)
print("\n--- list con condiciones ---")
df_filtrado = df_hh98[(df_hh98['sexhead'] == 0) & (df_hh98['agehead'] < 45)]
print(df_filtrado[['famsize', 'educhead']])

# Stata: codebook
# El comando `info()` y `describe()` de pandas cubren parte de esta funcionalidad
print("\n--- codebook (usando info y describe) ---")
print(df_hh98.info())
print(df_hh98.describe(include='all'))

# Stata: count
print("\n--- count ---")
print(f"Número total de observaciones: {len(df_hh98)}")

# Stata: count if agehead>50
print("\n--- count con condición ---")
count_mayor_50 = len(df_hh98[df_hh98['agehead'] > 50])
print(f"Número de jefes de hogar mayores de 50: {count_mayor_50}")

# Stata: summarize famsize educhead, d
print("\n--- summarize con detalle ---")
print(df_hh98[['famsize', 'educhead']].describe())

# Stata: by dfmfd: sum famsize educhead
print("\n--- summarize por grupo (by) ---")
print(df_hh98.groupby('dfmfd')[['famsize', 'educhead']].describe())

# Stata: tabstat famsize educhead, stats(mean sd) by(dfmfd)
print("\n--- tabstat (usando groupby y agg) ---")
print(df_hh98.groupby('dfmfd')[['famsize', 'educhead']].agg(['mean', 'std']))

# Stata: tab dfmfd
print("\n--- tabulate (tab) ---")
print(df_hh98['dfmfd'].value_counts())

# Stata: tab dfmfd sexhead, col row
print("\n--- tabla de contingencia ---")
tabla_cruzada = pd.crosstab(df_hh98['dfmfd'], df_hh98['sexhead'])
print(tabla_cruzada)
# Para porcentajes de fila (row)
print(pd.crosstab(df_hh98['dfmfd'], df_hh98['sexhead'], normalize='index'))

# Stata: histogram agehead
print("\n--- histograma ---")
plt.figure(figsize=(8, 6))
sns.histplot(df_hh98['agehead'], kde=True)
plt.title('Histograma de la edad del jefe de hogar')
plt.xlabel('Edad del jefe de hogar')
plt.ylabel('Frecuencia')
plt.show()

# Stata: twoway (scatter educhead agehead)
print("\n--- gráfico de dispersión (scatter plot) ---")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='agehead', y='educhead', data=df_hh98)
plt.title('Educación vs. Edad del jefe de hogar')
plt.xlabel('Edad del jefe de hogar')
plt.ylabel('Educación del jefe de hogar')
plt.show()

# Stata: correlate
print("\n--- correlación ---")
print(df_hh98[['famsize', 'educhead', 'agehead']].corr())

In [ ]:
# --- Replicando la Manipulación de Datos ---

# Stata: label variable oldhead "Jefe de Hogar mayor a 50: 1=Si 0=No"
# En pandas no hay una función directa para esto, pero se pueden usar comentarios en el código
# o un diccionario de metadatos.

# Stata: gen oldhead = 1 if agehead >50
# Stata: replace oldhead = 0 if agehead <=50
df_hh98['oldhead'] = np.where(df_hh98['agehead'] > 50, 1, 0)
print("\n--- Variable 'oldhead' creada ---")
print(df_hh98[['agehead', 'oldhead']].head())

# Stata: gen oldhead2 = [agehead>50] if agehead !=.
# Pandas maneja los valores nulos (NaN) automáticamente.
df_hh98['oldhead2'] = (df_hh98['agehead'] > 50).astype(int)
print("\n--- Variable 'oldhead2' creada ---")
print(df_hh98[['agehead', 'oldhead2']].head())

# Stata: egen avgagemf = mean(agehead), by(sexhead)
df_hh98['avgagemf'] = df_hh98.groupby('sexhead')['agehead'].transform('mean')
print("\n--- Variable 'avgagemf' creada con el promedio por sexo ---")
print(df_hh98[['sexhead', 'agehead', 'avgagemf']].head())

# Stata: keep if famsize <=6
df_hh98_keep = df_hh98[df_hh98['famsize'] <= 6].copy()
print(f"\nNúmero de observaciones después de 'keep if': {len(df_hh98_keep)}")

# Stata: drop famsize
df_hh98_drop = df_hh98.drop(columns=['famsize'])
print("\n--- Variable 'famsize' eliminada ---")
print(df_hh98_drop.columns)

# Stata: merge 1:1 nh using hh_98_2
# Simulación de la fusión de dos bases de datos
# Primero creamos dos DataFrames separados
df_1 = df_hh98[['nh', 'famsize', 'educhead']].copy()
df_2 = df_hh98[['nh', 'dfmfd', 'agehead']].copy()

# Ahora los fusionamos
df_merged = pd.merge(df_1, df_2, on='nh', how='inner')
print("\n--- Bases de datos fusionadas (merge) ---")
print(df_merged.head())

In [ ]:
# --- Replicando Macros y Loops ---

# Simulación de Stata 'globals'
# En Python, un diccionario puede simular un global
globals_stata = {}
globals_stata['control1'] = ['famsize', 'educhead']
globals_stata['control2'] = globals_stata['control1'] + ['agehead']

print("\n--- Simulación de 'globals' ---")
print(f"Variables en control1: {globals_stata['control1']}")
print(f"Variables en control2: {globals_stata['control2']}")

# Stata: sum $control2
print("\n--- Sumarización usando 'globals' ---")
print(df_hh98[globals_stata['control2']].describe())

# Stata: foreach var in per001 per011 per019
print("\n--- Bucle 'foreach' ---")
lista_vars = ['famsize', 'educhead', 'agehead']
for var in lista_vars:
    # `qui` se simula no imprimiendo nada
    media = df_hh98[var].mean()
    print(f"El promedio de la variable '{var}' es: {media:.2f}")